In [ ]:
# housing_regression.py
# A complete linear regression for the housing dataset

In [ ]:
# 1. Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# 2. Load the dataset
df = pd.read_csv('housing.csv')

In [ ]:
# 3. Initial inspection & cleaning
print("=== First five rows ===")
print(df.head(), "\n")

print("=== Data summary (numeric columns) ===")
print(df.describe().T, "\n")

print("=== Missing values per column ===")
print(df.isnull().sum(), "\n")

In [ ]:
# Figure 1 – Barplot of missing values before imputation
# ------------------------------------------------------
# Run this *before* the median fill step.

import matplotlib.pyplot as plt
import seaborn as sns

# Count NaNs column-wise
na_counts = df.isnull().sum()
na_counts = na_counts[na_counts > 0].sort_values() 

plt.figure(figsize=(8, 4))
sns.barplot(
    x=na_counts.values,
    y=na_counts.index,
    palette="crest",
    orient="h"
)
plt.title("Missing Values per Column (Pre-Imputation)")
plt.xlabel("Number of Missing Entries")
plt.ylabel("")
plt.tight_layout()
plt.show()


In [ ]:
# 3.1 Categorical encoding – one‑hot for ocean_proximity (required for OLS)
if "ocean_proximity" in df.columns:
    df = pd.get_dummies(df, columns=["ocean_proximity"], drop_first=True)
else:
    print("Note: 'ocean_proximity' already one‑hot encoded – skipping get_dummies.")

missing_total = df.isnull().sum().sum()
if missing_total > 0:
    num_cols = df.select_dtypes(include=[np.number]).columns
    df[num_cols] = df[num_cols].fillna(df[num_cols].median())
    print(f"Imputed {missing_total} missing numeric values with column medians.")
else:
    print("No missing numeric values to impute.")

In [ ]:
# 4. Exploratory Data Analysis (EDA) ───────────────────────────────────────
numeric_cols = df.select_dtypes(include=[np.number]).columns


In [ ]:
# 4.1 Histograms
_ = df[numeric_cols].hist(figsize=(14, 10), bins=30)
plt.suptitle("Histograms of Numeric Features", y=1.02, fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# 4.2 Boxplots
plt.figure(figsize=(14, 6))
sns.boxplot(data=df[numeric_cols], orient="h", palette="vlag")
plt.title("Boxplots of Numeric Features")
plt.tight_layout()
plt.show()

In [ ]:
# 4.3 Correlation heatmap
plt.figure(figsize=(12, 9))
corr = df[numeric_cols].corr()
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

In [ ]:
# 5. Outlier detection (report only, no removal) ───────────────────────────
Z = np.abs(stats.zscore(df[numeric_cols]))
outlier_mask = (Z >= 3).any(axis=1)
print(f"Rows with |Z| ≥ 3: {outlier_mask.sum()} (reported, not removed)\n")

In [ ]:
# 6. Train‑test split ──────────────────────────────────────────────────────
TARGET = "median_house_value"
X = df.drop(TARGET, axis=1)
y = df[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)


In [ ]:
#regression equation 

coef_series = pd.Series(linreg.coef_, index=X.columns)
eqn = "median_house_value = " + " + ".join(
    f"{b:.4f}*{c}" for c, b in coef_series.items()
) + f" + {linreg.intercept_:.4f}"
print("\nRegression equation:\n", eqn)


In [ ]:
# 7.1 scikit‑learn fit
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)

# 7.2 statsmodels OLS for detailed stats
X_train_sm = sm.add_constant(X_train).astype(float)
# Ensure y is float
y_train_float = y_train.astype(float)
ols_model = sm.OLS(y_train_float, X_train_sm).fit()
print(ols_model.summary())


In [ ]:
# 8. Diagnostics ───────────────────────────────────────────────────────────
# 8.1 VIF
vif = pd.DataFrame({
    "feature": X_train_sm.columns,
    "VIF": [variance_inflation_factor(X_train_sm.values, i) for i in range(X_train_sm.shape[1])]
})
print("\n=== Variance Inflation Factors ===")
print(vif, "\n")

# 8.2 Residual analysis
residuals = y_train_float - ols_model.predict(X_train_sm)

# Histogram
sns.histplot(residuals, kde=True)
plt.title("Residuals Distribution")
plt.show()

# Q-Q plot
sm.qqplot(residuals, line="45", fit=True)
plt.title("Q-Q Plot of Residuals")
plt.show()

# Residuals vs fitted
plt.scatter(ols_model.predict(X_train_sm), residuals, alpha=0.5)
plt.axhline(0, color="red", ls="--")
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.title("Residuals vs. Fitted")
plt.show()

# Normality & autocorrelation tests
sh_w, sh_p = stats.shapiro(residuals)
dw = sm.stats.stattools.durbin_watson(residuals)
print(f"Shapiro‑Wilk: W = {sh_w:.3f}, p = {sh_p:.4f}")
print(f"Durbin‑Watson : {dw:.3f}\n")

In [ ]:
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm

# --- 1.  Make column names safe for patsy (no spaces or punctuation) ---
safe_cols = {c: c.replace(" ", "_").replace(">", "gt") for c in df.columns}
df_ren = df.rename(columns=safe_cols)

# --- 2.  Build formula string ---
TARGET = "median_house_value"
predictors = [c for c in df_ren.columns if c != TARGET]
formula = TARGET + " ~ " + " + ".join(predictors)

# --- 3.  Fit formula-based OLS on *training* rows only (to keep parity) ---
train_idx = X_train.index                    # same rows used earlier
ols_formula = smf.ols(formula, data=df_ren.loc[train_idx]).fit()

# --- 4.  Print summary (optional) ---
print(ols_formula.summary())

# --- 5.  Type-II ANOVA table (matches sample report) ---
anova_tbl = anova_lm(ols_formula, typ=2)
print("\n=== ANOVA Table (Type II) ===\n", anova_tbl)


In [ ]:
# --- Type-II ANOVA on the training data -----------------------------------
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm

# 1) Make column names patsy-safe (replace spaces, symbols)
safe_map = {c: c.replace(" ", "_").replace("<", "lt").replace(">", "gt")
            for c in df.columns}
df_safe   = df.rename(columns=safe_map)

# 2) Build formula string:  median_house_value ~ all other columns
TARGET = "median_house_value"
predictors = [c for c in df_safe.columns if c != TARGET]
formula = TARGET + " ~ " + " + ".join(predictors)

# 3) Fit formula OLS **on training rows only**
train_idx   = X_train.index              # reuse earlier split
ols_formula = smf.ols(formula, data=df_safe.loc[train_idx]).fit()

# 4) Type-II sums of squares
anova_tbl = anova_lm(ols_formula, typ=2)   # columns: DF, sum_sq, F, PR(>F)
print("\n=== Type-II ANOVA Table ===\n")
print(anova_tbl.round(2))                  # <-- Table 4 for the report


In [ ]:
# ------------------------------------------
#  A)   Drop 'latitude' before VIF analysis
# ------------------------------------------
X_vif = X_train.drop(columns=['latitude'])   # keep longitude as proxy

# Cast to float
X_vif = X_vif.astype(float)

vif_df = pd.DataFrame({
    'feature': X_vif.columns,
    'VIF': [variance_inflation_factor(X_vif.values, i)
            for i in range(X_vif.shape[1])]
})

# Plot
plt.figure(figsize=(8, 5))
sns.barplot(data=vif_df.sort_values('VIF', ascending=False),
            x='VIF', y='feature', palette='mako')
plt.title('Variance Inflation Factors (latitude dropped)')
plt.tight_layout()
plt.show()

print(vif_df.sort_values('VIF', ascending=False))


In [ ]:
# 9. Performance metrics ─────────────────────────────────────────────────--
train_r2 = r2_score(y_train, ols_model.predict(X_train_sm))
test_r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("=== Model Performance ===")
print(f"R² (train): {train_r2:.4f}")
print(f"R² (test) : {test_r2:.4f}")
print(f"RMSE (test): {rmse:,.2f}\n")

In [ ]:
#Actual vs Predicted comparison table

compare_df = pd.DataFrame({
    "Actual": y_test.reset_index(drop=True),
    "Predicted": pd.Series(y_pred)
})
print(compare_df.head(20))


In [ ]:
#Accuracy %

accuracy_pct = compare_df.corr().iloc[0,1] * 100
print(f"Test-set accuracy ≈ {accuracy_pct:.1f}%")
